# Healthcare Analytics Dashboard

# Import Libraries

In [97]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Interactive widgets (optional)
import ipywidgets as widgets
from IPython.display import display

# Settings
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print(" All libraries imported successfully!")

 All libraries imported successfully!


In [98]:
# Load the data
df = pd.read_csv("healthcare_data.csv")

In [99]:
print("Data loaded successfully!")
print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

Data loaded successfully!
Shape: 20 rows × 12 columns


 # Initial Data Exploration

In [100]:
# Display first few rows
print("=" * 80)
print("FIRST 5 ROWS OF THE DATASET")
print("=" * 80)
display(df.head())

FIRST 5 ROWS OF THE DATASET


,Patient_ID,Age,Gender,Disease,Admission_Date,Discharge_Date,Hospital,City,Doctor,Treatment_Cost,Length_of_Stay,Outcome
0,P001,45,Male,Diabetes,2024-01-02,2024-01-08,Apollo Hospital,Hyderabad,Dr Rao,45000,6,Recovered
1,P002,32,Female,Covid-19,2024-01-05,2024-01-15,Fortis Hospital,Bangalore,Dr Mehta,78000,10,Recovered
2,P003,60,Male,Heart Disease,2024-01-07,2024-01-20,Apollo Hospital,Chennai,Dr Kumar,120000,13,Recovered
3,P004,28,Female,Malaria,2024-01-10,2024-01-14,AIIMS Delhi,Delhi,Dr Singh,15000,4,Recovered
4,P005,50,Male,Hypertension,2024-01-12,2024-01-18,Fortis Hospital,Mumbai,Dr Patel,40000,6,Recovered


In [101]:
# Dataset info
print("\n" + "=" * 80)
print("DATASET INFO")
print("=" * 80)
df.info()


DATASET INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Patient_ID      20 non-null     object
 1   Age             20 non-null     int64 
 2   Gender          20 non-null     object
 3   Disease         20 non-null     object
 4   Admission_Date  20 non-null     object
 5   Discharge_Date  20 non-null     object
 6   Hospital        20 non-null     object
 7   City            20 non-null     object
 8   Doctor          20 non-null     object
 9   Treatment_Cost  20 non-null     int64 
 10  Length_of_Stay  20 non-null     int64 
 11  Outcome         20 non-null     object
dtypes: int64(3), object(9)
memory usage: 2.0+ KB


In [102]:
# Statistical summary
print("\n" + "=" * 80)
print("STATISTICAL SUMMARY")
print("=" * 80)
display(df.describe())


STATISTICAL SUMMARY


,Age,Treatment_Cost,Length_of_Stay
count,20.000000,20.00000,20.000000
mean,44.850000,70850.00000,8.300000
std,13.487909,59669.50645,3.961658
min,26.000000,14000.00000,4.000000
25%,33.500000,25750.00000,5.750000
50%,43.500000,46000.00000,7.000000
75%,55.750000,88500.00000,10.750000
max,70.000000,210000.00000,16.000000


In [103]:
# Check for missing values
print("\n" + "=" * 80)
print("MISSING VALUES")
print("=" * 80); missing_values = df.isnull().sum()
display(missing_values[missing_values > 0])


MISSING VALUES


Series([], dtype: int64)

In [104]:
if missing_values.sum() == 0:
    print(" No missing values found!")

 No missing values found!


# Data Cleaning and Preprocessing

In [105]:
# Create a copy for cleaning
df_clean = df.copy()

In [106]:
# Clean column names (remove extra spaces)
df_clean.columns = df_clean.columns.str.strip()

In [107]:
# Convert date columns to datetime
df_clean['Admission_Date'] = pd.to_datetime(df_clean['Admission_Date'], errors='coerce')
df_clean['Discharge_Date'] = pd.to_datetime(df_clean['Discharge_Date'], errors='coerce')

In [108]:
# Calculate Length of Stay
df_clean['Length_of_Stay_Calc'] = (df_clean['Discharge_Date'] - df_clean['Admission_Date']).dt.days

In [109]:
# Extract admission month for time series analysis
df_clean['Admission_Month'] = df_clean['Admission_Date'].dt.to_period('M').astype(str)
df_clean['Admission_Year'] = df_clean['Admission_Date'].dt.year
df_clean['Admission_Month_Num'] = df_clean['Admission_Date'].dt.month

In [110]:
# Create age groups
df_clean['Age Group'] = pd.cut(
    df_clean['Age'],
    bins=[0, 18, 35, 50, 65, 80, 100],
    labels=['0-18', '19-35', '36-50', '51-65', '66-80', '80+']
)

In [111]:
# Remove rows with invalid dates
df_clean = df_clean.dropna(subset=['Admission_Date', 'Discharge_Date'])

In [112]:
print("Data cleaning completed!")

Data cleaning completed!


In [113]:
print(f"Clean dataset shape: {df_clean.shape[0]:,} rows × {df_clean.shape[1]} columns")

Clean dataset shape: 20 rows × 17 columns


In [114]:
print(f" Removed {len(df) - len(df_clean)} rows due to invalid dates")

 Removed 0 rows due to invalid dates


# Key Performance Indicators (KPIs)

In [115]:
print(df_clean.columns.tolist())

['Patient_ID', 'Age', 'Gender', 'Disease', 'Admission_Date', 'Discharge_Date', 'Hospital', 'City', 'Doctor', 'Treatment_Cost', 'Length_of_Stay', 'Outcome', 'Length_of_Stay_Calc', 'Admission_Month', 'Admission_Year', 'Admission_Month_Num', 'Age Group']


In [116]:
# Calculate KPIs
total_patients = len(df_clean)
total_revenue = df_clean['Treatment_Cost'].sum()
avg_billing = df_clean['Treatment_Cost'].mean()
avg_stay = df_clean['Length_of_Stay_Calc'].mean()

In [117]:
# Display KPIs in a nice format
print("=" * 80)
print(" HEALTHCARE ANALYTICS DASHBOARD - KEY PERFORMANCE INDICATORS")
print("=" * 80)

# KPI dictionary
kpi_data = {
    'Metric': [
        'Total Patients',
        'Total Revenue',
        'Average Billing Amount',
        'Average Length of Stay',
        'Number of Hospitals',
        'Number of Diseases'
    ],
    'Value': [
        f"{total_patients:,}",
        f"${total_revenue:,.2f}",
        f"${avg_billing:,.2f}",
        f"{avg_stay:.1f} days",
        df_clean['Hospital'].nunique(),
        df_clean['Disease'].nunique()
    ]
}

kpi_df = pd.DataFrame(kpi_data)
display(kpi_df)

 HEALTHCARE ANALYTICS DASHBOARD - KEY PERFORMANCE INDICATORS


,Metric,Value
0,Total Patients,20
1,Total Revenue,"$1,417,000.00"
2,Average Billing Amount,"$70,850.00"
3,Average Length of Stay,8.3 days
4,Number of Hospitals,5
5,Number of Diseases,8


# Patient Demographics Analysis

In [118]:
# Create subplots for demographics
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Age Distribution', 'Gender Distribution', 
                    'Age Group Distribution', 'Gender by Age Group'),
    specs=[[{'type': 'histogram'}, {'type': 'pie'}],
           [{'type': 'bar'}, {'type': 'bar'}]]
)

In [119]:
# 1) Age Distribution (Histogram)
fig.add_trace(
    go.Histogram(x=df_clean['Age'], nbinsx=20, name='Age', marker_color='lightblue'),
    row=1, col=1
)


In [120]:
# 2) Gender Distribution (Pie Chart)
gender_counts = df_clean['Gender'].value_counts()
fig.add_trace(
    go.Pie(labels=gender_counts.index, values=gender_counts.values, 
           name='Gender', hole=0.3),
    row=1, col=2
)

In [121]:
print(df_clean.columns.tolist())


['Patient_ID', 'Age', 'Gender', 'Disease', 'Admission_Date', 'Discharge_Date', 'Hospital', 'City', 'Doctor', 'Treatment_Cost', 'Length_of_Stay', 'Outcome', 'Length_of_Stay_Calc', 'Admission_Month', 'Admission_Year', 'Admission_Month_Num', 'Age Group']


In [122]:
# 3) Age Group Distribution
age_group_counts = df_clean['Age'].value_counts().sort_index()
fig.add_trace(
    go.Bar(x=age_group_counts.index, y=age_group_counts.values, 
           name='Age', marker_color='lightcoral'),
    row=2, col=1
)


In [123]:
# 4) Gender by Age Group
gender_age = df_clean.groupby(['Age', 'Gender']).size().reset_index(name='Count')
for gender in df_clean['Gender'].unique():
    data = gender_age[gender_age['Gender'] == gender]
    fig.add_trace(
        go.Bar(x=data['Age'], y=data['Count'], name=gender),
        row=2, col=2
    )

fig.update_layout(height=900, showlegend=True, title_text="Patient Demographics Analysis")
fig.show()

In [124]:
# Top 15 diseases
top_conditions = df_clean['Disease'].value_counts().nlargest(15)

# Create visualization
fig = px.bar(
    x=top_conditions.values,
    y=top_conditions.index,
    orientation='h',
    labels={'x': 'Number of Patients', 'y': 'Disease'},
    title='Top 15 Diseases',
    color=top_conditions.values,
    color_continuous_scale='Blues'
)
fig.show()


In [125]:
print("TOP 15 DISEASES - DETAILED ANALYSIS")
print("=" * 80)

condition_stats = (
    df_clean.groupby('Disease')
    .agg({
        'Treatment_Cost': ['mean', 'median', 'sum'],
        'Length_of_Stay': 'mean',
        'Age': 'mean',
        'Patient_ID': 'count'
    })
    .nlargest(15, ('Patient_ID', 'count'))
)

condition_stats.columns = [
    'Avg Billing', 'Median Billing', 'Total Revenue',
    'Avg Stay', 'Avg Age', 'Patient Count'
]

condition_stats = condition_stats.round(2)
print(condition_stats)


TOP 15 DISEASES - DETAILED ANALYSIS
               Avg Billing  Median Billing  Total Revenue  Avg Stay  Avg Age  \
Disease                                                                        
Covid-19          72666.67         72000.0         218000      9.33    36.00   
Hypertension      40333.33         40000.0         121000      6.33    52.00   
Heart Disease    133333.33        135000.0         400000     14.00    65.00   
Diabetes          47333.33         47000.0         142000      7.00    42.67   
Dengue            24000.00         24000.0          48000      5.50    32.00   
Cancer           205000.00        205000.0         410000     14.50    59.00   
Asthma            24500.00         24500.0          49000      4.00    37.00   
Malaria           14500.00         14500.0          29000      4.00    27.00   

               Patient Count  
Disease                       
Covid-19                   3  
Hypertension               3  
Heart Disease              3  
Diabetes

# Hospital Performance Analysis

In [126]:
# Hospital-wise metrics
hospital_stats = (
    df_clean.groupby('Hospital')
    .agg({
        'Patient_ID': 'count',
        'Treatment_Cost': ['mean', 'sum'],
        'Length_of_Stay': 'mean',
        'Age': 'mean'
    })
    .round(2)
)

hospital_stats.columns = [
    'Patient Count', 'Avg Billing', 'Total Revenue',
    'Avg Stay', 'Avg Age'
]

hospital_stats = hospital_stats.sort_values('Total Revenue', ascending=False)
print(hospital_stats)


                  Patient Count  Avg Billing  Total Revenue  Avg Stay  Avg Age
Hospital                                                                      
Tata Memorial                 2     205000.0         410000     14.50    59.00
Apollo Hospital               6      67000.0         402000      8.67    44.00
AIIMS Delhi                   4      77250.0         309000      9.25    47.25
Fortis Hospital               4      49750.0         199000      7.25    47.00
Manipal Hospital              4      24250.0          97000      4.75    34.50


In [127]:
# Visualization 1: Patient Count by Hospital
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Patient Count by Hospital', 'Average Billing by Hospital',
                    'Average Length of Stay by Hospital', 'Total Revenue by Hospital'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}]]
)


In [128]:
# Patient Count
fig.add_trace(
    go.Bar(x=hospital_stats.index, y=hospital_stats['Patient Count'],
           name='Patients', marker_color='lightblue'),
    row=1, col=1
)

In [129]:
# Average Billing
fig.add_trace(
    go.Bar(x=hospital_stats.index, y=hospital_stats['Avg Billing'],
           name='Avg Billing', marker_color='lightgreen'),
    row=1, col=2
)

In [130]:
# Average Stay
fig.add_trace(
    go.Bar(x=hospital_stats.index, y=hospital_stats['Avg Stay'],
           name='Avg Stay', marker_color='lightcoral'),
    row=2, col=1
)

In [131]:
# Total Revenue
fig.add_trace(
    go.Bar(x=hospital_stats.index, y=hospital_stats['Total Revenue'],
           name='Total Revenue', marker_color='gold'),
    row=2, col=2
)

fig.update_layout(height=900, showlegend=False, title_text="Hospital Performance Analysis")
fig.show()

# Time Series Analysis - Admission Trends

In [132]:
# Ensure Admission_Date is datetime
df_clean['Admission_Date'] = pd.to_datetime(df_clean['Admission_Date'], errors='coerce')

# Create Admission_Month column
df_clean['Admission_Month'] = df_clean['Admission_Date'].dt.to_period('M').astype(str)


In [133]:
admissions_over_time = df_clean.groupby('Admission_Month').agg({
    'Patient_ID': 'count',
    'Treatment_Cost': 'sum',
    'Length_of_Stay': 'mean'
}).reset_index()

admissions_over_time.columns = ['Month', 'Admissions', 'Revenue', 'Avg Stay']


In [134]:
# Sort by month
admissions_over_time = admissions_over_time.sort_values('Month')

# Create time series visualization
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Monthly Patient Admissions', 'Monthly Revenue Trend'),
    vertical_spacing=0.15
)


In [135]:
# Admissions line chart
fig.add_trace(
    go.Scatter(x=admissions_over_time['Month'], y=admissions_over_time['Admissions'],
               mode='lines+markers', name='Admissions', 
               line=dict(color='blue', width=2)),
    row=1, col=1
)


In [136]:
# Revenue bar chart
fig.add_trace(
    go.Bar(x=admissions_over_time['Month'], y=admissions_over_time['Revenue'],
           name='Revenue', marker_color='green'),
    row=2, col=1
)

fig.update_xaxes(title_text="Month", row=2, col=1)
fig.update_yaxes(title_text="Number of Patients", row=1, col=1)
fig.update_yaxes(title_text="Revenue ($)", row=2, col=1)
fig.update_layout(height=800, showlegend=True, title_text="Admission & Revenue Trends Over Time")
fig.show()

In [137]:
# Statistical insights
print("=" * 80)
print("ADMISSION TRENDS INSIGHTS")
print("=" * 80)
print(f" Peak Month: {admissions_over_time.loc[admissions_over_time['Admissions'].idxmax(), 'Month']}")
print(f" Patients: {admissions_over_time['Admissions'].max():.0f}")
print(f" Lowest Month: {admissions_over_time.loc[admissions_over_time['Admissions'].idxmin(), 'Month']}")
print(f" Patients: {admissions_over_time['Admissions'].min():.0f}")
print(f" Average Monthly Admissions: {admissions_over_time['Admissions'].mean():.1f}")
print(f" Total Revenue: ${admissions_over_time['Revenue'].sum():,.2f}")


ADMISSION TRENDS INSIGHTS
 Peak Month: 2024-01
 Patients: 10
 Lowest Month: 2024-01
 Patients: 10
 Average Monthly Admissions: 10.0
 Total Revenue: $1,417,000.00


# Billing Analysis

In [138]:
# Billing distribution
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Treatment Cost Distribution', 'Billing by Age Group',
                    'Billing by Gender', 'Billing by Hospital'),
    specs=[[{'type': 'histogram'}, {'type': 'bar'}],
           [{'type': 'box'}, {'type': 'bar'}]]
)

In [139]:
# 1) Histogram
fig.add_trace(
    go.Histogram(x=df_clean['Treatment_Cost'], nbinsx=50, name='Billing',
                 marker_color='lightblue'),
    row=1, col=1
)

In [140]:
# 2) Billing by Age Group
age_billing = df_clean.groupby('Age')['Treatment_Cost'].mean().round(2)
fig.add_trace(
    go.Bar(x=age_billing.index, y=age_billing.values, name='Age Group',
           marker_color='lightcoral'),
    row=1, col=2
)

In [141]:
# 3) Billing by Gender
gender_billing = df_clean.groupby('Gender')['Treatment_Cost'].mean().round(2)
fig.add_trace(
    go.Box(x=df_clean['Gender'], y=df_clean['Treatment_Cost'], name='Gender'),
    row=2, col=1
)


In [142]:
# 4) Billing by Hospital
hospital_billing = df_clean.groupby('Hospital')['Treatment_Cost'].mean().round(2).sort_values(ascending=False)
fig.add_trace(
    go.Bar(x=hospital_billing.index, y=hospital_billing.values, name='Hospital',
           marker_color='gold'),
    row=2, col=2
)

fig.update_layout(height=900, showlegend=False, title_text="Billing Analysis")
fig.show()

# Statistical summary
print("=" * 80)
print("BILLING STATISTICS")
print("=" * 80)
print(f"Minimum Bill: ${df_clean['Treatment_Cost'].min():,.2f}")
print(f"Maximum Bill: ${df_clean['Treatment_Cost'].max():,.2f}")
print(f"Average Bill: ${df_clean['Treatment_Cost'].mean():,.2f}")
print(f"Median Bill: ${df_clean['Treatment_Cost'].median():,.2f}")
print(f"Std Deviation: ${df_clean['Treatment_Cost'].std():,.2f}")


BILLING STATISTICS
Minimum Bill: $14,000.00
Maximum Bill: $210,000.00
Average Bill: $70,850.00
Median Bill: $46,000.00
Std Deviation: $59,669.51


# Correlation Analysis

In [143]:
# Select numeric columns for correlation
numeric_cols = df_clean[['Age', 'Treatment_Cost', 'Length_of_Stay']]

# If you prefer the calculated stay duration:
#numeric_cols = df_clean[['Age', 'Treatment_Cost', 'Length_of_Stay_Calc']]

# Calculate correlation matrix
correlation_matrix = numeric_cols.corr()

print(correlation_matrix)


                     Age  Treatment_Cost  Length_of_Stay
Age             1.000000        0.727498        0.729767
Treatment_Cost  0.727498        1.000000        0.925299
Length_of_Stay  0.729767        0.925299        1.000000


In [144]:
# Visualize correlation
fig = px.imshow(
    correlation_matrix,
    labels=dict(color="Correlation"),
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    color_continuous_scale='RdBu_r',
    aspect="auto",
    title="Correlation Matrix: Age, Billing Amount, and Length of Stay"
)
fig.show()

In [145]:
# Display correlation values
print("=" * 80)
print("CORRELATION ANALYSIS")
print("=" * 80)
display(correlation_matrix.round(3))

CORRELATION ANALYSIS


,Age,Treatment_Cost,Length_of_Stay
Age,1.000,0.727,0.730
Treatment_Cost,0.727,1.000,0.925
Length_of_Stay,0.730,0.925,1.000


In [146]:
fig = px.scatter(
    df_clean.sample(min(1000, len(df_clean))),  # sample up to available rows
    x='Age',
    y='Treatment_Cost',   # use correct column name
    title='Age vs Treatment Cost',
    trendline='ols',
    labels={'Age': 'Patient Age', 'Treatment_Cost': 'Treatment Cost ($)'}
)
fig.show()


 # Interactive Widgets

In [147]:
# Create interactive filters
def filter_and_display(hospital, condition, age_min, age_max):
    """Filter data and display statistics"""
    
    filtered_df = df_clean[
        (df_clean['Hospital'] == hospital) &
        (df_clean['Disease'] == condition) &   
        (df_clean['Age'] >= age_min) &
        (df_clean['Age'] <= age_max)
    ]
    
    print(f"\n{'='*80}")
    print("FILTERED RESULTS")
    print(f"{'='*80}")
    print(f"Hospital: {hospital}")
    print(f"Condition: {condition}")
    print(f"Age Range: {age_min}-{age_max}")
    print(f"\nTotal Patients: {len(filtered_df)}")
    print(f"Average Billing: ${filtered_df['Treatment_Cost'].mean():,.2f}")   # <-- use 'Treatment_Cost'
    print(f"Average Stay: {filtered_df['Length_of_Stay'].mean():.1f} days")   # <-- use 'Length_of_Stay'
    
    return filtered_df

# Get unique values for dropdowns
hospitals = df_clean['Hospital'].unique().tolist()
conditions = df_clean['Disease'].unique().tolist()[:10]  # <-- use 'Disease'

# Create widgets
hospital_widget = widgets.Dropdown(options=hospitals, description='Hospital:')
condition_widget = widgets.Dropdown(options=conditions, description='Condition:')
age_min_widget = widgets.IntSlider(min=0, max=100, value=0, description='Min Age:')
age_max_widget = widgets.IntSlider(min=0, max=100, value=100, description='Max Age:')

# Create interactive output
output = widgets.interactive_output(
    filter_and_display,
    {
        'hospital': hospital_widget,
        'condition': condition_widget,
        'age_min': age_min_widget,
        'age_max': age_max_widget
    }
)

# Display widgets
display(hospital_widget, condition_widget, age_min_widget, age_max_widget, output)


Dropdown(description='Hospital:', options=('Apollo Hospital', 'Fortis Hospital', 'AIIMS Delhi', 'Manipal Hospi…

Dropdown(description='Condition:', options=('Diabetes', 'Covid-19', 'Heart Disease', 'Malaria', 'Hypertension'…

IntSlider(value=0, description='Min Age:')

IntSlider(value=100, description='Max Age:')

Output()

# Key Insights and Recommendations

In [148]:
print("=" * 80)
print("KEY INSIGHTS & RECOMMENDATIONS")
print("=" * 80)

insights = f"""
1. PATIENT DEMOGRAPHICS
   • Total Patients Analyzed: {total_patients:,}
   • Gender Distribution: {gender_counts.to_dict()}
   • Most Common Age Group: {age_group_counts.idxmax()} ({age_group_counts.max()} patients)

2. DISEASE TRENDS
   • Most Common Condition: {top_conditions.index[0]} ({top_conditions.values[0]} patients)
   • Top 5 conditions account for {top_conditions[:5].sum()} patients ({top_conditions[:5].sum()/total_patients*100:.1f}% of total)

3. HOSPITAL PERFORMANCE
   • Highest Patient Volume: {hospital_stats['Patient Count'].idxmax()} ({hospital_stats['Patient Count'].max():.0f} patients)
   • Highest Average Billing: {hospital_stats['Avg Billing'].idxmax()} (${hospital_stats['Avg Billing'].max():,.2f})
   • Highest Total Revenue: {hospital_stats['Total Revenue'].idxmax()} (${hospital_stats['Total Revenue'].max():,.2f})

4. FINANCIAL INSIGHTS
   • Total Revenue: ${total_revenue:,.2f}
   • Average Billing per Patient: ${avg_billing:,.2f}
   • Revenue Range: ${df_clean['Treatment_Cost'].min():,.2f} - ${df_clean['Treatment_Cost'].max():,.2f}

5. OPERATIONAL METRICS
   • Average Length of Stay: {avg_stay:.1f} days
   • Peak Admission Month: {admissions_over_time.loc[admissions_over_time['Admissions'].idxmax(), 'Month']}
   • Average Monthly Admissions: {admissions_over_time['Admissions'].mean():.1f}

RECOMMENDATIONS:
   1. Focus resources on top 5 medical conditions (80% of cases)
   2. Review billing practices at hospitals with highest average charges
   3. Plan staffing based on peak admission months
   4. Investigate correlation between age and treatment cost
   5. Consider targeted preventive care for most common age groups
"""

print(insights)


KEY INSIGHTS & RECOMMENDATIONS

1. PATIENT DEMOGRAPHICS
   • Total Patients Analyzed: 20
   • Gender Distribution: {'Male': 10, 'Female': 10}
   • Most Common Age Group: 26 (1 patients)

2. DISEASE TRENDS
   • Most Common Condition: Diabetes (3 patients)
   • Top 5 conditions account for 14 patients (70.0% of total)

3. HOSPITAL PERFORMANCE
   • Highest Patient Volume: Apollo Hospital (6 patients)
   • Highest Average Billing: Tata Memorial ($205,000.00)
   • Highest Total Revenue: Tata Memorial ($410,000.00)

4. FINANCIAL INSIGHTS
   • Total Revenue: $1,417,000.00
   • Average Billing per Patient: $70,850.00
   • Revenue Range: $14,000.00 - $210,000.00

5. OPERATIONAL METRICS
   • Average Length of Stay: 8.3 days
   • Peak Admission Month: 2024-01
   • Average Monthly Admissions: 10.0

RECOMMENDATIONS:
   1. Focus resources on top 5 medical conditions (80% of cases)
   2. Review billing practices at hospitals with highest average charges
   3. Plan staffing based on peak admission mon

# Export Analysis Results

In [149]:
import os

# Ensure data folder exists
os.makedirs("data", exist_ok=True)

# Export cleaned dataset
df_clean.to_csv('data/healthcare_data_cleaned.csv', index=False)
print(" -->Cleaned data exported to 'data/healthcare_data_cleaned.csv'")

# Export summary statistics
if 'insights' in locals():
    with open('analysis_summary.txt', 'w') as f:
        f.write("HEALTHCARE ANALYTICS DASHBOARD - SUMMARY REPORT\n")
        f.write("=" * 80 + "\n\n")
        f.write(insights)
    print(" -->Summary report exported to 'analysis_summary.txt'")
else:
    print("'insights' variable not defined. Run insights block first.")

 -->Cleaned data exported to 'data/healthcare_data_cleaned.csv'
 -->Summary report exported to 'analysis_summary.txt'
